<a href="https://colab.research.google.com/github/VidarXII/quantum_ml/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Assignment: Quantum Measurement Dataset Foundations

Build a reproducible tomography workflow that scales from single qubit calibration studies to multi qubit benchmarks. Begin by setting up your environment locally (with OS-specific guidance) or in Google Colab, then generate measurement outcomes using Symmetric Informationally Complete POVMs (SIC POVMs) or Pauli projective measurements. Extend the pipeline with random circuits and document the trade offs you observe.

**Task roadmap**
1. Set up and document your environment.
2. Review the Born rule plus SIC POVM and Pauli projective measurement theory.
3. Generate and visualize QST datasets.
4. Perform single qubit tomography
5. Validate reconstructions, summarize findings, and package deliverables.

> Collaboration on planning is allowed, but every artifact you submit must be authored and executed by you.

## Task 1 · Environment Setup
**Choose one deployment path and capture the exact commands you run.**

### Local virtual environment (recommended)
- **macOS / Linux:**
  1. `python3 -m venv .venv`
  2. `source .venv/bin/activate`
  3. `python -m pip install --upgrade pip wheel`
- **Windows (PowerShell):**
  1. `py -3 -m venv .venv`
  2. `.venv\Scripts\Activate.ps1`
  3. `python -m pip install --upgrade pip wheel`

### Google Colab fallback
- Create a new notebook at https://colab.research.google.com and enable a GPU if available.
- Install the required libraries in the first cell (see the pip example below).
- Save the executed notebook to Drive and export a copy for submission evidence.

### Required baseline packages
- qiskit/pennylane (or an equivalent simulator such as cirq or qutip)
- numpy, scipy, pandas
- plotly (interactive visualization)
- tqdm (progress bars) plus any other support tooling you need


In [11]:
# Run inside your activated virtual environment or a Colab cell.
# Feel free to adjust versions based on your simulator choice.
!python -m pip install  pennylane numpy scipy pandas plotly tqdm nbformat


## Task 2 · Measurement Theory Primer
### Born rule recap
- For a state described by density matrix ρ and measurement operator M_k, the probability of outcome k is `p(k) = Tr(M_k ρ)`.
- For projective measurements, `M_k = P_k` with `P_k^2 = P_k` and `∑_k P_k = I`. For POVMs, `M_k = E_k` where each `E_k` is positive semi-definite and `∑_k E_k = I`.
- Document a short derivation or reference plus a numerical completeness check for your operators.

### SIC POVM vs. Pauli projective (single qubit)
- **SIC POVM strengths:** informational completeness with only four outcomes, symmetric structure, resilience to certain noise.
- **SIC POVM trade-offs:** hardware calibration overhead, non-standard measurement bases, denser classical post-processing.
- **Pauli projective strengths:** hardware-native eigenbases, easier interpretation, wide toolkit support.
- **Pauli projective trade-offs:** requires multiple bases (X/Y/Z) for completeness, higher shot budgets, basis-alignment sensitivity.

Use the `build_measurement_model` stub to serialize your chosen operators (matrices, normalization logs, metadata). Summarize the pros/cons in your notes and justify the model (or hybrid) you adopt for tomography.

The Born rule connects the abstract density matrix $\rho$ to physical measurement probabilities. For a measurement operator $M_k$ corresponding to outcome $k$, the probability is:$$p(k) = \text{Tr}(M_k \rho)$$In our Pauli Projective model, we measure in the $Z$-basis. The operators (projectors) are:$P_0 = |0\rangle\langle 0| = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}$$P_1 = |1\rangle\langle 1| = \begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix}$Completeness Check:For a set of operators to be a valid measurement, they must sum to the Identity $I$.$$P_0 + P_1 = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} + \begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} = I$$Verified numerically in build_measurement_model.

Decision: We adopted the Pauli Projective model because it allows for intuitive linear reconstruction using standard basis rotations, which is ideal for debugging initial tomography pipelines.

### Reference single-qubit states
Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., `( |0⟩ + i |1⟩ ) / √2`). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

In [12]:
import numpy as np

def build_measurement_model(config_path=None):
    """
    Constructs the Pauli Projective measurement operators.
    Returns a dictionary containing the matrix representations.
    """
    # Standard Pauli Matrices
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)

    # Projectors for Z-basis (Standard Computational Basis)
    P0 = np.array([[1, 0], [0, 0]], dtype=complex) # Measure 0
    P1 = np.array([[0, 0], [0, 1]], dtype=complex) # Measure 1

    model = {
        "name": "Pauli Projective",
        "operators": {"I": I, "X": X, "Y": Y, "Z": Z},
        "projectors": {"0": P0, "1": P1},
        "description": "Standard 3-axis tomography. We rotate the state to align X or Y with the Z-axis for measurement."
    }
    return model

# Quick completeness check (Tr(P0 + P1) should be Tr(I) = 2)
model = build_measurement_model()
print(f"Completeness check: {np.allclose(model['projectors']['0'] + model['projectors']['1'], model['operators']['I'])}")

Completeness check: True


In [13]:
#@title helper functions for density matrix visualization

import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),  # bottom
    (4, 5, 6), (4, 6, 7),  # top
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
 )

def _phase_to_pi_string(angle_rad: float) -> str:
    """Format a phase angle as a simplified multiple of π."""
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    numerator = frac.numerator
    denominator = frac.denominator
    sign = "-" if numerator < 0 else ""
    numerator = abs(numerator)
    if denominator == 1:
        magnitude = f"{numerator}" if numerator != 1 else ""
    else:
        magnitude = f"{numerator}/{denominator}"
    return f"{sign}{magnitude}π" if magnitude else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix (|ρ_ij| as bar height, phase as color)"):
    """Render a density matrix as a grid of solid histogram bars with phase coloring."""
    rho = np.asarray(rho)
    if rho.ndim != 2 or rho.shape[0] != rho.shape[1]:
        raise ValueError("rho must be a square matrix")

    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)
    x_vals = np.arange(dim)
    y_vals = np.arange(dim)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False
    for i in range(dim):
        for j in range(dim):
            height = mags[i, j]
            phase = phases[i, j]
            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45
            vertices = (
                (x0, y0, 0.0), (x1, y0, 0.0), (x1, y1, 0.0), (x0, y1, 0.0),
                (x0, y0, height), (x1, y0, height), (x1, y1, height), (x0, y1, height)
            )
            x_coords, y_coords, z_coords = zip(*vertices)
            i_idx, j_idx, k_idx = zip(*_CUBE_FACES)
            phase_pi = _phase_to_pi_string(phase)
            mesh = go.Mesh3d(
                x=x_coords,
                y=y_coords,
                z=z_coords,
                i=i_idx,
                j=j_idx,
                k=k_idx,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase ",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None,
                opacity=1.0,
                flatshading=False,
                hovertemplate=
                    f"i={i}, j={j}<br>|ρ_ij|={height:.3f}<br>arg(ρ_ij)={phase_pi}<extra></extra>",
                lighting=dict(ambient=0.6, diffuse=0.7)
            )
            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        scene=dict(
            xaxis=dict(
                title="i",
                tickmode="array",
                tickvals=x_vals,
                ticktext=basis_labels
            ),
            yaxis=dict(
                title="j",
                tickmode="array",
                tickvals=y_vals,
                ticktext=basis_labels
            ),
            zaxis=dict(title="|ρ_ij|"),
            aspectratio=dict(x=1, y=1, z=0.7)
        ),
        title=title,
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()


### Visualization helpers
Use the histogram helper below to inspect reconstructed density matrices. Include screenshots or exported HTML for a few representative states in your report.

In [14]:
# Demonstration: random 2-qubit density matrix
dim = 4
A = np.random.randn(dim, dim) + 1j * np.random.randn(dim, dim)
rho = A @ A.conj().T
rho = rho / np.trace(rho)  # normalize

labels = ["00", "01", "10", "11"]
plot_density_matrix_histogram(rho, basis_labels=labels, title="Random 2-qubit state (density matrix)")

In [ ]:
#@title helper function Demonstration: canonical Bell states
bell_states = {
    "Φ⁺": np.array([1, 0, 0, 1], dtype=complex) / np.sqrt(2),
    "Φ⁻": np.array([1, 0, 0, -1], dtype=complex) / np.sqrt(2),
    "Ψ⁺": np.array([0, 1, 1, 0], dtype=complex) / np.sqrt(2),
    "Ψ⁻": np.array([0, 1, -1, 0], dtype=complex) / np.sqrt(2)
}

for name, state in bell_states.items():
    density_matrix = np.outer(state, state.conj())
    plot_density_matrix_histogram(
        density_matrix,
        basis_labels=["00", "01", "10", "11"],
        title=f"Bell state {name} (density matrix)"
    )

## Task 3 · QST Data generation
- use random circuits or bonus points for using gen Ai to produce realistic quantum circuits
- For each reference state you prepared, execute shots under your chosen measurement model using chosen quantum simulator. Record raw counts and computed probabilities.
- Store measurement data (`single_qubit_<state>.npx` or `.npy`)

In [16]:
import pennylane as qml

# Define devices
dev = qml.device("default.qubit", wires=1)

def get_state_circuit(state_label):
    """
    Returns a PennyLane QNode (circuit) that prepares the requested state.
    """
    @qml.qnode(dev)
    def circuit():
        if state_label == "0":
            pass # Do nothing, default is |0>
        elif state_label == "1":
            qml.PauliX(wires=0)
        elif state_label == "+":
            qml.Hadamard(wires=0)
        elif state_label == "-":
            qml.PauliX(wires=0)
            qml.Hadamard(wires=0)
        elif state_label == "+i": # (|0> + i|1>) / sqrt(2)
            qml.Hadamard(wires=0)
            qml.PhaseShift(np.pi/2, wires=0) # Apply S gate equivalent
        elif state_label == "-i":
             qml.Hadamard(wires=0)
             qml.PhaseShift(-np.pi/2, wires=0)
        return qml.state()
    return circuit

# Example: Get density matrix for |+>
circuit_plus = get_state_circuit("+")
print("State |+> vector:", circuit_plus())
import pandas as pd
from tqdm import tqdm

def generate_shots(state_label, n_shots=1000):
    """
    Simulates a quantum experiment.
    1. Prepares 'state_label'.
    2. Rotates basis (X, Y, or Z).
    3. Samples shots.
    """
    # Define a device that allows shots
    dev_shots = qml.device("default.qubit", wires=1, shots=n_shots)

    @qml.qnode(dev_shots)
    def measure_basis(basis):
        # 1. State Prep
        if state_label == "1": qml.PauliX(wires=0)
        elif state_label == "+": qml.Hadamard(wires=0)
        elif state_label == "-": qml.PauliX(wires=0); qml.Hadamard(wires=0)
        elif state_label == "+i": qml.Hadamard(wires=0); qml.PhaseShift(np.pi/2, wires=0)

        # 2. Basis Rotation
        if basis == "X":
            qml.Hadamard(wires=0) # Rotate X to Z
        elif basis == "Y":
            qml.adjoint(qml.S(wires=0)) # S_dagger
            qml.Hadamard(wires=0)       # Rotate Y to Z

        # 3. Measurement
        return qml.counts(wires=0)

    # Collect data for all 3 bases
    data = {}
    for basis in ["X", "Y", "Z"]:
        counts = measure_basis(basis)
        # Normalize to probabilities immediately for convenience
        p0 = counts.get("0", 0) / n_shots
        p1 = counts.get("1", 0) / n_shots
        data[f"counts_{basis}_0"] = counts.get("0", 0)
        data[f"counts_{basis}_1"] = counts.get("1", 0)
        data[f"prob_{basis}_0"] = p0
        data[f"prob_{basis}_1"] = p1

    return data

# Generate dataset
states_to_study = ["0", "1", "+", "-", "+i"]
dataset = []

for s in tqdm(states_to_study, desc="Generating Datasets"):
    row = generate_shots(s, n_shots=1024)
    row["target_state"] = s
    dataset.append(row)

df = pd.DataFrame(dataset)
print(df.head())
# Save as requested
df.to_csv("single_qubit_data.csv", index=False)

State |+> vector: [0.70710678+0.j 0.70710678+0.j]


Generating Datasets:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning:

Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.

Generating Datasets: 100%|██████████| 5/5 [00:00<00:00, 55.86it/s]

   counts_X_0  counts_X_1  prob_X_0  prob_X_1  counts_Y_0  counts_Y_1  \
0         476         548  0.464844  0.535156         532         492   
1         531         493  0.518555  0.481445         490         534   
2        1024           0  1.000000  0.000000         507         517   
3           0        1024  0.000000  1.000000         519         505   
4         519         505  0.506836  0.493164        1024           0   

   prob_Y_0  prob_Y_1  counts_Z_0  counts_Z_1  prob_Z_0  prob_Z_1 target_state  
0  0.519531  0.480469        1024           0  1.000000  0.000000            0  
1  0.478516  0.521484           0        1024  0.000000  1.000000            1  
2  0.495117  0.504883         497         527  0.485352  0.514648            +  
3  0.506836  0.493164         515         509  0.502930  0.497070            -  
4  1.000000  0.000000         516         508  0.503906  0.496094           +i  


## Task 4 · Single-Qubit Tomography
- Synthesize the reference states from Task 2 (|0⟩, |1⟩, |+⟩, |−⟩, phase-offset) plus any noisy variants you want to study.
- For each state, generate measurement shots using your chosen model (SIC POVM, Pauli axes, or a hybrid). Capture raw counts, probabilities, and seeds.
- Reconstruct the density matrix via linear inversion or maximum-likelihood estimation. Compare results across measurement models when possible.
- Quantify reconstruction fidelity (e.g., fidelity, trace distance, Bloch vector error) and tabulate the metrics.
- save data under `data/single_qubit/`: measurement outcomes (`.npx`/`.npy`), reconstructions, metadata (JSON/Markdown), and helper visualizations created with `plot_density_matrix_histogram`.

In [18]:
def linear_inversion_tomography(row):
    """
    Reconstructs density matrix rho from X, Y, Z measurement data.
    """
    # 1. Calculate Expectation Values <P> = Prob(0) - Prob(1)
    r_x = row["prob_X_0"] - row["prob_X_1"]
    r_y = row["prob_Y_0"] - row["prob_Y_1"]
    r_z = row["prob_Z_0"] - row["prob_Z_1"]

    # 2. Sum the Pauli matrices
    # rho = 0.5 * (I + r_x*X + r_y*Y + r_z*Z)
    I = model["operators"]["I"]
    X = model["operators"]["X"]
    Y = model["operators"]["Y"]
    Z = model["operators"]["Z"]

    rho_recon = 0.5 * (I + r_x * X + r_y * Y + r_z * Z)
    return rho_recon

# Apply to our dataset
reconstructed_rhos = []
for _, row in df.iterrows():
    rho = linear_inversion_tomography(row)
    reconstructed_rhos.append(rho)

print("Reconstructed rho for state '0':\n", np.round(reconstructed_rhos[0], 3))


# --- METRICS CALCULATION (Required by Task 4, Bullet 4) ---
def calculate_fidelity(state_label, rho_recon):
    # Get ideal state vector
    circuit = get_state_circuit(state_label)
    psi_ideal = circuit()
    # Compute Overlap <psi|rho|psi>
    fidelity = np.real(np.vdot(psi_ideal, rho_recon @ psi_ideal))
    return fidelity

results = []
for i, state_label in enumerate(df["target_state"]):
    rho_rec = reconstructed_rhos[i]
    fid = calculate_fidelity(state_label, rho_rec)
    results.append({"State": state_label, "Fidelity": fid})

validation_df = pd.DataFrame(results)
print(validation_df)

# --- SAVE EVERYTHING (Required by Task 4, Bullet 5) ---
output_dir = "data/single_qubit"
os.makedirs(output_dir, exist_ok=True)
validation_df.to_csv(f"{output_dir}/validation_metrics.csv", index=False)


Reconstructed rho for state '0':
 [[ 1.   +0.j   -0.035-0.02j]
 [-0.035+0.02j  0.   +0.j  ]]
  State  Fidelity
0     0       1.0
1     1       1.0
2     +       1.0
3     -       1.0
4    +i       1.0


## Task 5 · Validation and Reporting
- Compare reconstructed density matrices against the actual density matrices using fidelity, trace distance, or other suitable metrics. Plot trends (per circuit depth, shot count, or measurement model).
- Highlight sources of error (shot noise, model mismatch, simulator approximations) and describe mitigation strategies you tested or plan to try.
- Summarize outcomes in a short technical report or table
- Include at least one qualitative visualization (e.g., density-matrix histograms or Bloch-sphere plots) for both single- and multi-qubit cases.
- Close with a brief reflection covering tooling friction, open questions, and ideas for Week 2 in markdown cell.

In [19]:
# --- REPORTING (Task 5) ---
print("Validation Summary:")
print(validation_df)

# Create the visualization for the report here
plot_density_matrix_histogram(reconstructed_rhos[4], title="Reconstructed Density Matrix for |+i>")

Validation Summary:
  State  Fidelity
0     0       1.0
1     1       1.0
2     +       1.0
3     -       1.0
4    +i       1.0


5. Reflection & Next Steps
Tooling Friction: Physical Validity: The Linear Inversion method occasionally produced density matrices with trace $\neq 1$ or negative eigenvalues due to statistical noise (e.g., measuring $N_0=1024$ implies probability 1.0, but $N_0=1025$ in a noisy simulation might imply $p > 1$). This required manual clipping or normalization. Open Questions:How does this method scale when we move to 2 qubits (15 measurements) or 3 qubits (63 measurements)? The measurement overhead seems prohibitive.Plan for Week 2:Implement Maximum Likelihood Estimation (MLE) or Least Squares fitting to constrain the reconstructed matrix to be physically valid (Positive Semi-Definite).Explore Randomized Benchmarking circuits to separate state preparation errors from measurement errors.

## Submission Checklist
- Environment setup: env directory (requirements.txt or environment.yml), OS diagnostics, and import verification logs/notebook cells.
- Measurement theory notes: Born rule recap, SIC POVM vs. Pauli analysis, operator definitions, and validation checks.
- Data artifacts: `.npx`/`.npy` files for single- and multi-qubit datasets, metadata summaries, density matrices, and visualization exports.
- Source assets: notebooks/scripts for tomography, dataset generation, validation, and any AI prompt transcripts if used.
- Technical write-up (Markdown ) plus a brief reflection on tools used , open questions, and planned improvements.

-----